# Modified Diffusion Entropy Analysis 

Garland Culbreth, Jacob Baxley, David Lambert.

Center for Nonlinear Science, University of North Texas.

Last update: 2021-05-22

## Introduction
---

This notebook includes functions for performing both MDEA (DEA with stripes) and the original DEA without. The functions with stripes were written by G.C. with contribution from J.B. The original (no stripe) DEA function was done by J.B. and D.L., and vectorized by G.C.

This is a major update of the DEA program using Python. It's loosely based on an older Python conversion of my even older MATLAB version, which was converted by Jacob Baxley, David Lambert, and me. That conversion relied on an old module of legacy code, which was outdated and in need of refinement. For this version I wrote a completely new function that performs the same procedure but faster and using standard Python/NumPy functions.

### Using this notebook

 - All work should be done in the bottom code cell, the one labeled "Work cell". 
 - Useage instructions and documentation etc. can be found in markdown cells or function docstrings.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from tqdm import tqdm

In [ ]:
# plt.style.use('ggplot')
sns.set_theme(context='notebook', style='ticks', palette='deep', font_scale=1.3)

## Sample data
---

Generator for a simple Random Walk, for testing purposes. If you have no data use this to generate some. Also provides a baseline to test against.

In [ ]:
def sample_data(length):
    """Generates an array of sample data."""
    np.random.seed(1010)  # for baseline consistency
    random_steps = np.random.choice([-1, 1], length)
    random_steps[0] = 0  # always start from 0
    random_walk = np.cumsum(random_steps)
    return random_walk

In [ ]:
some_data = sample_data(10000)

f, ax = plt.subplots(figsize=(5, 4))
ax.plot(some_data)
ax.set_xlabel('t')
ax.set_ylabel('x(t)')
ax.grid(True)
sns.despine(left=True, bottom=True)
plt.show()

## Functions
---

In [ ]:
def apply_stripes(data, stripes, show_plot): 
    """
    Rounds `data` to `stripes` evenly spaced intervals.

    Parameters
    ----------
    data : array_like
        Time-series data to be examined.
    stripes : int
        Number of stripes to apply. 
    show_plot : bool
        If True, show data plot with overlaid stripes.

    Returns
    ----------
    rounded_data : ndarray
        `data` rounded to `stripes` number of equally spaced intervals.
    """
    
    if show_plot == True:
        lines = np.linspace(min(data), max(data), num=stripes)
        plt.figure(figsize=(5, 4))
        plt.plot(data)
        plt.hlines(y=lines, xmin=0, xmax=len(data), colors='0.3', linewidths=1, alpha=0.4)
        plt.xlabel('t')
        plt.ylabel('Data(t)')
        plt.title('Data with stripes')
        sns.despine()
        plt.show()
    
    if min(data) <= 0:
        data = data + abs(min(data))
    elif min(data) > 0:
        data = data - abs(min(data))
    max_data = max(data)
    min_data = min(data)
    data_width = abs(max_data - min_data)
    stripe_size = data_width / stripes
    rounded_data = data / stripe_size
    return rounded_data

In [ ]:
def find_events(series):
    """Records an event (1) when `series` changes value."""
    events = []
    for i in range(1, len(series)):
        if (series[i] < np.floor(series[i-1])+1 and 
            series[i] > np.ceil(series[i-1])-1):
            # if both true, no crossing
            events.append(0)
        else:
            events.append(1)
    np.append(events, 0)
    return events

In [ ]:
def make_trajectory(events):
    """Constructs diffusion trajectory from events."""
    trajectory = np.cumsum(events)
    return trajectory

In [ ]:
def entropy(trajectory):
    """
    Calculates the Shannon Entropy of the diffusion trajectory.

    Generates a range of window lengths L. Steps each one along 
    'trajectory' and computes the displacement of 'trajectory' 
    over each window position. Bins these displacements, and divides 
    by the sum of all bins to make the probability distribution 'p'. 
    Puts 'p' into the equation for Shannon Entropy to get s(L).
    Repeats for all L in range 'window_lengths'.

    Parameters
    ----------
    trajectory : array_like
        Diffusion trajectory. Constructed by make_trajectory.

    Returns
    ----------
    s : ndarray
        Shannon Entropy values, S(L).
    window_lengths : ndarray
        Window lengths, L. 

    Notes
    ----------
    'tqdm(...)' makes the progress bar appear.
    """
    S = []
    window_lengths = np.arange(1, int(0.25*len(trajectory)), 1)
    for L in tqdm(window_lengths):
        window_starts = np.arange(0, len(trajectory)-L, 1)
        window_ends = np.arange(L, len(trajectory), 1)
        displacements = trajectory[window_ends] - trajectory[window_starts]
        bin_counts = np.bincount(displacements)
        bin_counts = bin_counts[bin_counts != 0]
        P = bin_counts / np.sum(bin_counts)
        S.append(-np.sum(P * np.log(P)))
    return S, window_lengths

In [ ]:
def no_stripe_entropy(trajectory):
    """
    Calculates the Shannon Entropy of the diffusion trajectory.

    Generates a range of window lengths L. Steps each one along 
    'trajectory' and computes the displacement of 'trajectory' 
    over each window position. Bins these displacements, and divides 
    by the sum of all bins to make the probability distribution 'p'. 
    Puts 'p' into the equation for Shannon Entropy to get s(L).
    Repeats for all L in range 'window_lengths'.

    Parameters
    ----------
    trajectory : array_like
        Diffusion trajectory. FOR NO STRIPES JUST PASS THE DATA SERIES.

    Returns
    ----------
    S : ndarray
        Shannon Entropy values, S(L).
    window_lengths : ndarray
        Window lengths, L.

    Notes
    ----------
    `tqdm()` makes the progress bar appear.
    """
    window_lengths = np.arange(1, int(0.25*len(trajectory)), 1)
    S = []
    for L in tqdm(window_lengths):
        window_starts = np.arange(0, len(trajectory)-L, 1)
        window_ends = np.arange(L, len(trajectory), 1)
        traj = trajectory[window_starts] - trajectory[window_ends]
        counts, bin_edge = np.histogram(traj, bins='doane')  # doane least bad for nongaussian
        counts = np.array(counts[counts != 0])
        binsize = bin_edge[1] - bin_edge[0]
        P = counts / sum(counts)
        S.append(-sum(P*np.log(P)) + np.log(binsize))
    return S, window_lengths

In [ ]:
def get_scaling(S, L, start, stop):
    """
    Calculates scaling.
    
    Calculates the scaling of the time-series by performing a 
    least-squares linear fit over S(l) and ln(l).

    Parameters
    ----------
    s : array_like
        Shannon Entropy values. 
    L : array_like
        Window Lengths. 
    start : int
        Index at which to start the fit slice.
    stop : int
        Index at which to stop the fit slice.

    Returns
    ----------
    L_slice : ndarray 
        The slice of window lengths L.
    coefficients : ndarray
        Slope and intercept of the fit. 

    Notes
    ----------
    Least-squares linear fits on log scale data have issues, 
    see doi:10.1371/journal.pone.0085777
    Making a version that uses the powerlaw package instead 
    would be better...
    """
    S_slice = S[start:stop]
    L_slice = L[start:stop]
    coefficients = np.polyfit(np.log(L_slice), S_slice, 1)
    return L_slice, coefficients

In [ ]:
def get_mu(delta):
    """
    Calculates the mu.

    Parameters
    ----------
    delta : float
        Scaling of the time-series process. 

    Returns
    ----------
    mu : float
        Complexity parameter. Powerlaw index for inter-event 
        time distribution.
    Notes
    ----------
    mu is calculated by both rules. later both are plotted
    against the line relating delta and mu, to hopefully
    let users graphically determine the correct mu.
    """
    mu1 = 1 + delta
    mu2 = 1 + (1 / delta)
    return mu1, mu2

In [ ]:
def plot_results(L, S, x_interval, slope, y_intercept, mu):
    """testing a plotting function"""
    fig, ax = plt.subplots()
    ax.plot(L, S, linestyle='', marker='.')
    ax.plot(x_interval, slope * np.log(x_interval) + y_intercept, color='k',
             label='$\delta = $'+str(np.round(slope, 2)))
    ax.plot([], [], linestyle='',label='$\mu = $'+str(np.round(mu, 2)))
    return ax

In [ ]:
def plot_mu_candidates(delta, mu1, mu2):
    x1 = np.linspace(1, 2, 100)
    x2 = np.linspace(2, 3, 100)
    x3 = np.linspace(3, 4, 100)
    y1 = x1 - 1
    y2 = 1 / (x2 - 1)
    y3 = np.full(100, 0.5)

    plt.figure(figsize=(5, 4))
    plt.plot(x1, y1, color='k')
    plt.plot(x2, y2, color='k')
    plt.plot(x3, y3, color='k')
    plt.plot(mu1, delta,
             marker='o',
             label='$\\mu$ = '+str(np.round(mu1, 2)))
    plt.plot(mu2, delta,
             marker='o',
             label='$\\mu$ = '+str(np.round(mu2, 2)))  
    plt.xticks(ticks=np.linspace(1, 4, 7))
    plt.yticks(ticks=np.linspace(0, 1, 5))
    plt.xlabel('$\\mu$')
    plt.ylabel('$\\delta$')
    plt.legend(loc=0)
    plt.grid(True)
    sns.despine(left=True, bottom=True)
    # plt.show()
    return None

In [ ]:
def dea_no_stripes(data, start, stop):
    """
    Applies DEA without the stripes refinement.

    Original DEA. Takes the original time series as the diffusion 
    trajectory.

    Parameters
    ----------
    data : array_like
        Time-series to be analysed.
    start : int
        Array index at which to start linear fit.
    stop : int 
        Array index at which to stop linear fit.

    Returns
    ----------
    figure 
        A figure plotting S(l) vs. ln(l), overlaid with the fit 
        line, labelled with the scaling and mu values.
    """
    S, L = no_stripe_entropy(data)
    fit = get_scaling(S, L, start, stop)
    mu = get_mu(fit[1][0])

    plt.figure(figsize=(5, 4))
    plt.plot(L, S, linestyle='', marker='.')
    plt.plot(fit[0], fit[1][0] * np.log(fit[0]) + fit[1][1], color='k',
             label='$\delta = {}$'.format(np.round(fit[1][0], 2)))
    plt.plot([], [], linestyle='', 
             label='$\mu = {}$'.format(np.round(mu, 2)))
    plt.xscale('log')
    plt.xlabel('$ln(l)$')
    plt.ylabel('$S(l)$')
    plt.legend(loc=0)
    plt.grid(True)
    sns.despine(left=True, bottom=True)
    # plt.show()
    return None

In [ ]:
def dea_with_stripes(data, stripes, start, stop, data_plot):
    """
    Applies DEA with the stripes refinement.

    Runs a sequence of functions to apply stripes and then 
    perform DEA on the data series. 

    Parameters
    ----------
    data : array_like
        Time-series to be analysed.
    stripes : int
        Number of stripes to be applied to the data.
    start : int
        Array index at which to start linear fit.
    stop : int 
        Array index at which to stop linear fit.
    data_plot : bool
        If True, show data plot with overlaid stripes.

    Returns
    ----------
    fig : figure 
        A figure plotting S(l) vs. ln(l), overlaid with the fit 
        line, labelled with the scaling and mu values.
    """
    rounded_data = apply_stripes(data, stripes, data_plot)
    event_array = find_events(rounded_data)
    diffusion_trajectory = make_trajectory(event_array)
    s, L = entropy(diffusion_trajectory)
    fit = get_scaling(s, L, start, stop)
    mu = get_mu(fit[1][0])

    plt.figure(figsize=(5, 4))
    plt.plot(L, s, linestyle='', marker='.')
    plt.plot(fit[0], fit[1][0] * np.log(fit[0]) + fit[1][1], color='k',
             label='$\delta = $'+str(np.round(fit[1][0], 2)))
    plt.xscale('log')
    plt.xlabel('$ln(L)$')
    plt.ylabel('$S(L)$')
    plt.legend(loc=0)
    # plt.grid(True, which='major')
    # plt.grid(True, which='minor', linewidth=0.4)
    sns.despine()
    # plt.show()

    plt.figure(figsize=(5, 4))
    check_mus_plot = plot_mu_candidates(fit[1][0], mu[0], mu[1])
    return None

## Work Cell
---

Everything you need to work with is in this cell.  

### Running  
 - Load your data into the `data` variable. Numpy arrays or Pandas DataFrame columns will work.  
 - Choose the number of stripes to apply, and the interval (slice) over which to fit.  
 - Run!  
 - Adjust `number_of_stripes` and slicing indexes, `fit_start` and `fit_stop`, as necessary.  

### Output  
 - A progress bar will display progress and time elapsed/remaining.  
 - On completion, two figures will be drawn with the results:
   - One will be of $S(l)$ vs. $ln(l)$, with the slope fit overlaid. This is the main result.
   - The other will be a display of the two possible values for $\mu$, drawn on a graph of the analytical behavior of $\delta$ vs. $\mu$. This is to help you determine which $\mu$ value is right.
 - Interpretation of results has its own section, below.  

### Choosing number of stripes  
 - Run a few initial tests with different values. e.g. 10, 50, 100.  
 - If there is significant disagreement in the scalings measured, vary about those trial numbers.  
 - The correct number of stripes to use is that number such that when varied up or down a little, the scaling does not change.
 - Rigorous rules for this are still being developed...  

### Choosing fit interval  
 - In the result figure, if the results are good, there will be a region in the loglog figure that appears linear. You want the fit interval to line up with this region.  
 - $S(l)$ is logged in calculation, $l$ is logged by the scale of the plot. The fitting function accounts for this.  

In [ ]:
### ----WORK HERE---- ###
data = sample_data(10000)
number_of_stripes = 40  # needs to be at least 2
fit_start = 30
fit_stop = 500
show_data_plot = False  # make True to see plot of data with stripes

result = dea_with_stripes(data, number_of_stripes, fit_start, fit_stop, show_data_plot)

### Without stripes

In [ ]:
# no_stripe_result = dea_no_stripes(data, start=50, stop=500)
# plt.show()

## Interpreting Results
---

### Basics  
 - $\delta$ is the measured scaling of the time-series process.  
 - $\mu$ is a complexity index, defined as the power for the inter-event time distribution $1/\tau^{\mu}$ ($\tau$ being inter-event time).  

### Baselines  
 - For a totally random process, DEA yields $\delta = 0.5.$  
 - The closer $\delta$ is to 1, and the closer $\mu$ is to 2, the more complex the data-series is. Those are the critical values of $\delta$ and $\mu$.  
 - If $\delta < 0.5$, this usually means the time-series is not complex. At least, not in this sense. 

### Determining $\mu$  
 - Two ways of calculating $\mu$ are employed:  
   - For $1 < \mu < 2$: $\mu = 1 + \delta$.  
   - For $2 < \mu < 3$: $\mu = 1 + 1/\delta$.  
 - The correct calculation for $\mu$ varies. As rigorous rules for determining which is correct in what situation have yet to be laid down, both candidates are calculated and plotted so that the user may compare them. Typically the correct value for $\mu$ will lie along the line, as the line represents the theoretical relationship.
 - If you already have an expectation for what range $\mu$ should be in, e.g. from theoretical arguments, use that.
 - The theoretical justifications for the two methods of deriving $\mu$ from the scaling $\delta$ are given in Section 3.2 of Reference 1 and Section 3.1-3.2 of Reference 3.

## Advanced use
---


### Speed and runtime
In the function `entropy()` the range of window lengths $l$ is defined such that each is 1 larger than the previous:  
        
        window_lengths = np.arange(1, int(0.25*len(trajectory)), 1)  

This can make the program take a long time to run if you give it a long time-series (e.g. 20000 timesteps takes ~1 second). If you want to speed up the runtime, you can increase the step size for the range, e.g.:

        window_lengths = np.arange(1, int(0.25*len(trajectory)), 10)  

This will still give good results, for long time-series, but can greatly speed up runtime.

Doing this also means you will have fewer values of $S(l)$ and $ln(l)$ to work with though, so be mindful.


### Maximum window length
In the function `entropy()` the range of window lengths ends at 0.25 times the length of the diffusion trajectory:

        window_lengths = np.arange(1, int(0.25*len(trajectory)), 10)  

You can increase further if you want, but there is usually no need. For longer window lengths $l$, and especially for $l$ longer than `0.5*len(trajectory)`, $S(l)$ quickly starts to decay and doesn't give anything useful. 

## References
---
 1. Culbreth, G., West, B. J., & Grigolini, P. (2019). Entropic approach to the detection of crucial events. *Entropy*, *21*(2), 178. [doi:10.3390/e21020178](https://doi.org/10.3390/e21020178)
    - This paper introduced the stripes and describes their role with figure examples.
    - Note: this paper used $\eta$ to denote scaling, rather than $\delta$.  

 2. Scafetta, N., & Grigolini, P. (2002). Scaling detection in time series: diffusion entropy analysis. *Physical Review E*, *66*(3), 036130. [doi:10.1103/PhysRevE.66.036130](https://doi.org/10.1103/PhysRevE.66.036130)
    - The actual algorithm for DEA is detailed (mostly only in words) in Section IV.

 3. Grigolini, P., Palatella, L., & Raffaelli, G. (2001). Asymmetric anomalous diffusion: an efficient way to detect memory in time series. *Fractals*, *9*(04), 439-449. [doi:10.1142/S0218348X01000865](https://doi.org/10.1142/S0218348X01000865)
    - This paper introduced always using positive steps when constructing the event array, rather than the sign of the step at that time index.